This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'THRn3DvwaQvDQ8TbXtby'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
from collections import defaultdict,OrderedDict, namedtuple
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# I am collecting data from the FSE database (FSE stands for Frankfurt Stock Exchange),
# of the stock AFX_X, from 08-01-2017 to 08-02-2017.
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params=dict(key=API_KEY,database_code='FSE',dataset_code='AFX_X',start_date='2017-08-01',end_date='2017-08-02')
r=requests.get(url,params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

json_data=r.json()
print(json_data)

# The JSON structure is indeed very  nested thoug hthe main dictionary has only
# one key which is the nested dictionary. I will keep the structure as is
# because there is no instructions about "beheading the first key". 
#{'dataset_data': 
#          {'limit': None, 
#           'transform': None,
#           'column_index': None, 
#           'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 
#           'start_date': '2017-08-01', 
#           'end_date': '2017-08-02', 
#           'frequency': 'daily', 
#           'data': [
#                   ['2017-08-02', 45.25, 45.77, 44.9, 45.56, None, 187468.0, 8528548.0, None, None, None],
#                   ['2017-08-01', 45.24, 45.54, 45.1, 45.45, None, 74975.0, 3399891.0, None, None, None]], 
#           'collapse': None, 
#           'order': None}}


{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-08-01', 'end_date': '2017-08-02', 'frequency': 'daily', 'data': [['2017-08-02', 45.25, 45.77, 44.9, 45.56, None, 187468.0, 8528548.0, None, None, None], ['2017-08-01', 45.24, 45.54, 45.1, 45.45, None, 74975.0, 3399891.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# (Q1) Collecting the data for the whole year 2017

url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params=dict(key=API_KEY,database_code='FSE',dataset_code='AFX_X',start_date='2017-01-01',end_date='2017-12-31')
r=requests.get(url,params)

In [6]:
# (Q2) Convert the return JSON object into a Python dictionary

# getting my response in json_data
json_data=r.json()

# initiating an empty python dictionary
afx_dict={}

# filling the dictionary afx_dict with the content of json_data

for key,value in json_data.items():
    afx_dict.update({key:value})
print(afx_dict.keys())
print(afx_dict['dataset_data'].keys())


dict_keys(['dataset_data'])
dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [7]:
# (Q3) Calculate what the highest and lowest opening prices were for the stock in this period.

# I extract the list called 'data' which is nested in 'dataset_data' 
# I store this sublist in the list afx_data.
afx_data=afx_dict['dataset_data']['data']

# I create an empty list 'open_prices' which I will use in the loop that follows
open_prices=[]

# The loop iterates through the lists (sublists) of list(afx_data)
# The loop skips "None" values
# The loop appends float values to the list open_prices
# I could have used a list comprehension, however, I won't because I have to in Q7
for sublist in afx_data:
        for i in sublist:
            if sublist[1]!=None:
                open_prices.append(sublist[1])
            else:
                pass

# I am printing out the minimum and maximum opening prices            
print('The highest opening price was {} and the lowest opening price was {} for the AFX_X stock in 2017.'.format(max(open_prices),min(open_prices)))

# The highest opening price was 53.11 and the lowest opening price was 34.0 for the AFX_X stock in 2017

The highest opening price was 53.11 and the lowest opening price was 34.0 for the AFX_X stock in 2017.


In [8]:
# (Q4) What was the largest change in any one day (based on High and Low price)?

# I am recycling afx_data from Q3 I am going to calculate 'High'-'low', 
# store it in the list 'change_list' and find the max

# Initiating the list 'change_list'
change_list=[]

# The loop iterates through the lists (sublists) of list(afx_data)
# The loop skips "None" values under both 'High' and 'Low'
# The loop appends the difference between 'High' and 'Low' to the list 'change_list'
# I could have used a list comprehension, however, I won't because I have to in Q7

for sublist in afx_data:
        for i in sublist:
            if (sublist[2]!=None) | (sublist[3]!=None):
                change=sublist[2]-sublist[3]
                change_list.append(change)
            else:
                pass

# I am printing out the largest change in any one day
print('The largest change in any one day was {} for the AFX_X stock in 2017.'.format(round(max(change_list),2)))

# The largest change in any one day was 2.81 for the AFX_X stock in 2017.

The largest change in any one day was 2.81 for the AFX_X stock in 2017.


In [9]:
# (Q5) What was the largest change between any two days (based on Closing Price)?

# I am recycling afx_data from Q3/4 I am going to subset the 'Close' data in the list 'close_list'
# Then I will calculate 'Close(today)- CLose(yesterday)' from close_list',
# store it in the list 'any_two' and find the max

# Initiating the list 'any_two' and 'close_list'
any_two=[]
close_list=[]

# The loop iterates through the lists (sublists) of list(afx_data)
# The loop skips "None" values under 'Close'
# The loop appends the 'data from 'Close' into the list 'close_list'

for sublist in afx_data:
        for i in sublist:
            if (sublist[4]!=None):
                close_list.append(sublist[4])
            else:
                pass
            
# The second loop goes through the list 'close_list', 
# starting on day 2 (index=1) and it appends the difference between
# the next value (i) and the precedent value(i-1) to the list "any_two"            
for i in range(1,len(close_list)):
        any_two.append(close_list[i]-close_list[i-1])
        
# I am printing out the largest change in any two days
print('The largest change in any two days was {} for the AFX_X stock in 2017.'.format(round(max(any_two),2)))

# The largest change in any two days was 2.56 for the AFX_X stock in 2017.

The largest change in any two days was 2.56 for the AFX_X stock in 2017.


In [16]:
# (Q6) What was the average daily trading volume during this year?

# Initiating the list 'trading_vol'
trading_vol=[]

# The loop iterates through the lists (sublists) of list(afx_data)
# The loop skips "None" values under 'trading_value'
# The loop appends the 'data from 'trading value' into the list 'trading_vol'

for sublist in afx_data:
        for i in sublist:
            if (sublist[6]!=None):
                trading_vol.append(sublist[6])
            else:
                pass

# I am printing out the mean trading volume
# I would have used the statistics.mean() but it seems we need to avoid using packages
# so I make a basic mean calculation sum/length
print('The average daily trading volume was {} for the AFX_X stock in 2017.'.format(round((sum(trading_vol)/len(trading_vol)),2)))

# The average daily trading volume was 89124.34 for the AFX_X stock in 2017.


The average daily trading volume was 89124.34 for the AFX_X stock in 2017.
2805


In [56]:
# (Q7) (Optional) What was the median trading volume during this year.
# (Note: you may need to implement your own function for calculating the median.)



# median_fn is my function to find the median using a comprehension list
# the function expects a 'dico' parameter which is here the python dictionary 
# with the only key a.k.a afx_dict['dataset_data']
# the function also expects a parameter col_name which is the name of the data 
# we want to utilize to calculate the median. The col_name is found under 'column_names'.
# for this exercise col_name is 'Traded Volume'.

def median_fn(dico,col_name):
    # The first subfunction "which data" looks for the position of the data in the list
    # by using its header name. Headers are stored in the list 'column_names'. 
    # In this exercise, the header is "Traded Volume' and can be found
    # in the list 'column_names' at position 6. The index of col_name
    # in 'column_names' is returned (would be 6 for 'Traded Volume')
    def whichdata(dic,col):
        return dic['column_names'].index(col)
     
    # The second subfunction uses a comprehension list to build a list with 
    # the data we have chosen to look at. I chose to filter out 'None' in case
    # I would like to use the function on other data list which contained 'None' instead of a value
    # The list is then sorted and finally is returned.
    def list_data(subindex):
        zelist=[sublist[subindex] for sublist in dico['data'] if sublist[subindex]!='None']
        zelist.sort()
        return zelist
    
    # The third subfunction uses if/else statement to discriminate
    # odd vs. even length of list in order to use the proper calculation
    # for the median. The function returns the median.
    def findmedian(alist):
         if not len(alist)%2:
            return alist[len(alist)//2-1]
         else:
            return ((alist[len(alist)//2 - 2] + alist[len(alist)//2 -1]) / 2)
        
    # The main function median_fn returns the median using its three nested functions
    return findmedian(list_data(whichdata(dico,col_name)))

# I am printing out the result
print('The median trading volume for AFX_X in 2017 is {} .'.format(median_fn(afx_dict['dataset_data'],'Traded Volume')))


# The median trading volume for AFX_X in 2017 is 74723.5 .




The median trading volume for AFX_X in 2017 is 74723.5 .
